In [3]:
#!pip install --upgrade mxnet==1.6.0
#!pip install gluonts

In [3]:
import pandas as pd
import matplotlib.pyplot as mpl
import matplotlib.pyplot as plt
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
import numpy as np

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


In [26]:
df = pd.read_csv('mv_port_processed_clusters.csv')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (16,46) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project Number'] = df['Project Number'].astype('str') 

In [28]:
df=df[df['Project Number'] != 'fw0labor00in'] 
df=df[df['Project Number'] != 'fw0labor0out'] 

In [29]:
df_sub = df[['Date', 'Project Number', 'Actual Net Revenue - MTD']]

In [30]:
df_pivot=df_sub.assign(d=df_sub.Date, t=df_sub['Project Number']) \
            .pivot_table(index='d', columns='t', values='Actual Net Revenue - MTD', fill_value=0)
            #.rename_axis(None, 1)

In [31]:
df_pivot = df_pivot.reset_index()

In [32]:
df_pivot=df_pivot.T

In [33]:
df_pivot = df_pivot.iloc[1:,]

In [96]:
df_pivot

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
t,,,,,,,,,,,,,,,,,,,,,
000221760018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000221760021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000221760022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000221760050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000221762018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zz000zzzfs20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zz000zzzim20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zz000zzzis20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
## for deep AR, we need to extract the specifc variables

In [97]:
columnlist = [i for i in range(0,63)]

In [98]:
df_pivot.columns = columnlist

In [99]:
#df_pivot.reset_index()

In [100]:
df_pivot.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62],
           dtype='int64')

In [101]:
ts_code = df_pivot.index.astype('category').codes
ts_code

array([    0,     1,     2, ..., 78123, 78124, 78125], dtype=int32)

In [102]:
total=len(df_pivot.columns)
ind = int(total*0.82)

ind
df_train = df_pivot.iloc[:, :ind].values
df_test = df_pivot.iloc[:, ind:].values

In [103]:
df_train.shape

(78126, 51)

In [104]:
len(df_test)

78126

In [105]:
freq ='1M'
start_train = pd.Timestamp('2016-01-01', freq=freq)
start_test = pd.Timestamp('2020-04-01', freq=freq)
prediction_length = 12

In [110]:
#tune this

estimator = DeepAREstimator(freq=freq, 
                           context_length=51,
                           use_feat_static_cat=True,
                           cardinality=[1],
                           num_layers=2, 
                           num_cells=32, 
                           cell_type ='lstm', trainer=Trainer(epochs=25, learning_rate=0.001), prediction_length=prediction_length)

In [172]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

train_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_train,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_train[32700:32750],
                            ts_code[32700:32750].reshape(-1,1))
], freq=freq)


In [173]:
test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_test,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_test[32700:32750],
                            ts_code[32700:32750].reshape(-1,1))
], freq=freq)



In [174]:
next(iter(train_ds))

{'target': array([  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         75.96, 282.86,  72.71], dtype=float32),
 'start': Timestamp('2016-01-31 00:00:00', freq='M'),
 'feat_static_cat': array([32700], dtype=int32),
 'source': SourceContext(source='list_data', row=0)}

In [155]:
df_train[1000:1200]

array([[ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ..., -3.995000e+01,
         0.000000e+00,  1.410000e+01],
       ...,
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         2.629000e+03,  4.888400e+03],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  2.001704e+04,
         2.844000e+02,  2.302000e+01]])

In [175]:
predictor = estimator.train(training_data=train_ds)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:02<00:00, 17.48it/s, epoch=25/25, avg_epoch_loss=-21.3]


In [114]:
from gluonts.evaluation.backtest import make_evaluation_predictions

In [181]:
forecast_it, ts_it = make_evaluation_predictions( dataset=test_ds, 
                                                predictor=predictor,
                                                num_samples=50)

In [182]:
from tqdm.autonotebook import tqdm

print('Obtaining time series conditioning values...')
tss = list(tqdm(ts_it, total=len(df_test)))
print('Obtaining time series predictions...')
forecasts = list(tqdm(forecast_it, total=len(df_test)))

Obtaining time series conditioning values...


  0%|          | 0/78126 [00:00<?, ?it/s]

Obtaining time series predictions...


  0%|          | 0/78126 [00:00<?, ?it/s]

In [183]:
forecasts = list(forecast_it)
tss = list(ts_it)

In [187]:
forecast_entry = forecasts[0]

IndexError: list index out of range

In [146]:

print(f"Number of sample paths: {forecast_entry.num_samples}")
print(f"Dimension of samples: {forecast_entry.samples.shape}")
print(f"Start date of the forecast window: {forecast_entry.start_date}")
print(f"Frequency of the time series: {forecast_entry.freq}")

Number of sample paths: 200
Dimension of samples: (200, 12)
Start date of the forecast window: 2020-04-30 00:00:00
Frequency of the time series: 1M


In [ ]:
ts_entry = tss[20]

In [147]:
print(f"Mean of the future window:\n {forecast_entry.mean}")
print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.5)}")

Mean of the future window:
 [ 8.3252598e-12 -7.1752446e-13 -6.2791378e-12 -2.0096904e-12
 -3.0824143e-12 -4.5928088e-12 -4.1888927e-12 -2.3330890e-12
  1.5686980e-12  2.1386892e-12  1.7715432e-12 -1.1666965e-12]
0.5-quantile (median) of the future window:
 [ 1.4147417e-12 -2.4251003e-12 -5.6632108e-12 -7.3757303e-13
 -2.9321530e-12 -3.1610597e-12 -6.2168309e-12 -4.8892357e-12
 -1.4503172e-12 -4.9459498e-13 -5.4901468e-16 -3.3853181e-12]


In [ ]:

def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = 150
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] #+ [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()

In [ ]:
plot_prob_forecasts(ts_entry, forecast_entry)

In [138]:
df_forecast = pd.DataFrame.from_records(prediction_list)

In [139]:
df_forecast

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [35]:
%matplotlib inline

def plot_prob_forecasts(ts_entry, forecast_entry):
    #pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
    
    plot_length = prediction_length
    prediction_intervals = (80.0, 95.0)
    legend = ['observations', 'median prediction'] + [ f"{k}% prediction interval" for k in prediction_intervals][::-1]
    
    fig, ax = plt.subplots(1,1, figsize(15, 7))
    ts_entry[-plot_length:].plot(ax=ax)
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which='both')
    plt.legend(legend, loc='upper left')
    plt.show()
    

In [117]:
for i in tqdm(range(200)):
    ts_entry = tss[i]
    forecast_entry = forecasts[i]
    plot_prob_forecasts(ts_entry, forecast_entry)

  0%|          | 0/200 [00:00<?, ?it/s]

NameError: name 'plot_prob_forecasts' is not defined

In [129]:
tss

[                      0
 2020-04-30   662.869995
 2020-05-31 -7886.080078
 2020-06-30   607.590027
 2020-07-31   703.489990
 2020-08-31  3893.840088
 2020-09-30  1877.400024
 2020-10-31   -63.560001
 2020-11-30   -44.900002
 2020-12-31  4403.220215
 2021-01-31  6986.379883
 2021-02-28 -7475.549805
 2021-03-31   222.619995,
                 0
 2020-04-30  237.5
 2020-05-31    0.0
 2020-06-30    0.0
 2020-07-31    0.0
 2020-08-31    0.0
 2020-09-30    0.0
 2020-10-31    0.0
 2020-11-30    0.0
 2020-12-31    0.0
 2021-01-31    0.0
 2021-02-28    0.0
 2021-03-31    0.0,
                         0
 2020-04-30    1169.540039
 2020-05-31  162641.156250
 2020-06-30       0.000000
 2020-07-31       0.000000
 2020-08-31       0.000000
 2020-09-30 -240761.875000
 2020-10-31   -2792.120117
 2020-11-30   20607.000000
 2020-12-31   23521.119141
 2021-01-31  -82674.000000
 2021-02-28  200723.453125
 2021-03-31       0.000000,
                      0
 2020-04-30     0.33000
 2020-05-31    -0.76000
 2

In [124]:
forecasts[2]

gluonts.model.forecast.SampleForecast(freq="1M", info=None, item_id=None, samples=numpy.array([[1.3569672295543622e-11, 9.241836462781094e-12, -1.5780210471660894e-11, 3.496790704216046e-11, 3.5771878081208852e-12, -9.807036259468216e-12, 1.988880934944426e-11, -2.3739073207185157e-11, -1.478384775710495e-11, 1.1396068984315111e-11, 5.343223693360377e-12, 1.2449869260522384e-11], [2.8973991608727268e-11, 1.0859268345653206e-11, 3.933047984516769e-11, -9.078039708843022e-11, -1.7502744045772012e-11, 4.771005639170323e-12, 5.307729516318416e-12, 2.3310239155582835e-11, -2.5568911137668904e-12, 5.50172450999864e-12, 1.0371478849358073e-11, 2.677408468543252e-11], [-1.1808171627991637e-11, 5.565459985229504e-12, 1.639433901134968e-11, 1.8081538870340363e-12, 1.7891883721116164e-12, 9.02667090380005e-12, 5.696093336587937e-12, -1.7914741955518676e-12, -9.156272962052014e-12, -4.006126680389244e-11, -3.299713627336054e-11, 1.0153318463768102e-10], [-1.8025225409501466e-11, 2.1898342514314884

In [121]:
df_predict = pd.DataFrame.from_records(forecasts)

TypeError: 'SampleForecast' object is not iterable

In [136]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(df_test[2000:2500]))

Running evaluation: 100%|██████████| 200/200 [00:00<00:00, 3031.77it/s]/Users/julia/opt/anaconda3/lib/python3.7/site-packages/gluonts/evaluation/metrics.py:80: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs(target - forecast)) / seasonal_error
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/gluonts/evaluation/metrics.py:80: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs(target - forecast)) / seasonal_error
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/gluonts/evaluation/metrics.py:80: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs(target - forecast)) / seasonal_error
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/gluonts/evaluation/metrics.py:157: RuntimeWarning: divide by zero encountered in true_divide
  return numerator / seasonal_error
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/gluonts/evaluation/metrics.py:157: RuntimeWarning: divide by zero 

In [138]:
item_metrics

,item_id,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,MAPE,sMAPE,OWA,MSIS,QuantileLoss[0.1],Coverage[0.1],QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.9],Coverage[0.9]
0,NaN,1.431623e-25,4.472779e-12,0.000000,0.000000,NaN,NaN,NaN,2.0,NaN,NaN,9.915420e-13,0.000000,4.472779e-12,1.000000,2.648395e-12,1.000000
1,NaN,1.459811e-25,4.359194e-12,0.000000,0.000000,NaN,NaN,NaN,2.0,NaN,NaN,1.038047e-12,0.000000,4.359194e-12,1.000000,2.782576e-12,1.000000
2,NaN,1.593526e-25,4.602894e-12,0.000000,0.000000,NaN,NaN,NaN,2.0,NaN,NaN,9.453924e-13,0.000000,4.602894e-12,1.000000,2.776123e-12,1.000000
3,NaN,1.416693e-25,4.571188e-12,0.000000,0.000000,NaN,NaN,NaN,2.0,NaN,NaN,1.023591e-12,0.000000,4.571188e-12,1.000000,2.818505e-12,1.000000
4,NaN,1.436620e-25,4.472986e-12,0.000000,0.000000,NaN,NaN,NaN,2.0,NaN,NaN,9.396931e-13,0.000000,4.472986e-12,1.000000,2.746012e-12,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,NaN,1.370126e+07,1.679316e+04,16793.160156,1399.430013,NaN,NaN,1.0,2.0,NaN,NaN,2.226718e+04,0.083333,1.679316e+04,0.916667,1.131914e+04,0.916667
196,NaN,9.201506e+02,1.050800e+02,105.080002,8.756667,NaN,NaN,1.0,2.0,NaN,NaN,2.101600e+01,0.000000,1.050800e+02,0.916667,1.891440e+02,0.916667
197,NaN,7.052143e+05,2.909050e+03,2909.050049,242.420837,NaN,NaN,1.0,2.0,NaN,NaN,5.236290e+03,0.083333,2.909050e+03,1.000000,5.818100e+02,1.000000
198,NaN,6.750000e+04,9.000000e+02,900.000000,75.000000,NaN,NaN,1.0,2.0,NaN,NaN,1.800000e+02,0.000000,9.000000e+02,0.916667,1.620000e+03,0.916667


In [145]:
mses=item_metrics.MSE
mse = mses.sum()/200

In [146]:
y_truth=df_test[1000:1200]

In [147]:
import numpy as np
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))
print('The normalized Root Mean Squared Error of our forecasts is {}'.format( (round(np.sqrt(mse), 2))/ ((y_truth.max() - y_truth.min())* 100)))

The Mean Squared Error of our forecasts is 53396825.18
The Root Mean Squared Error of our forecasts is 7307.31
The normalized Root Mean Squared Error of our forecasts is 0.00017155041471322223


In [149]:
0.00017155041471322223 * 100

0.017155041471322224

### we now use interpolation and exetrnal variables

In [34]:
sub = df_pivot.iloc[30000:30050, ]

In [35]:
df_pivot.T

t,000221760018,000221760021,000221760022,000221760050,000221762018,000231710001,000275410000,00041053mp81,000430560000,000771780000,...,zz000yyyis10,zz000yyylc10,zz000yyynt10,zz000zzzad20,zz000zzzae20,zz000zzzfs20,zz000zzzim20,zz000zzzis20,zz000zzzlc20,zz000zzznt20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
dfdate=df.groupby('Date')['Month'].count().reset_index()
dfdate.pop('Month')

0      2146
1      2252
2      2351
3      2438
4      2558
      ...  
58    48080
59    49329
60    50659
61    52503
62    54050
Name: Month, Length: 63, dtype: int64

In [50]:
backpivot=df_pivot.T
backpivot

t,000221760018,000221760021,000221760022,000221760050,000221762018,000231710001,000275410000,00041053mp81,000430560000,000771780000,...,zz000yyyis10,zz000yyylc10,zz000yyynt10,zz000zzzad20,zz000zzzae20,zz000zzzfs20,zz000zzzim20,zz000zzzis20,zz000zzzlc20,zz000zzznt20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
from statsmodels.tsa.stattools import adfuller
from math import sqrt

from scipy.stats import boxcox
from matplotlib import pyplot

df_forecast = []

for i in range(1,78126):
    diff = []
    b = i-1
    dft=backpivot.iloc[ :, b:i]
    df_new=pd.concat([dfdate, dft], axis=1)
    column=df_new.columns.values[1]
    
    df_new['Date'] = pd.to_datetime(df_new['Date'], format='%Y%m') 
    df_new=df_new.set_index('Date')
    upsampled=df_new.resample('D')
    interpolated=upsampled.interpolate(method='linear')
    interpolated=interpolated.values
    interpolated = np.concatenate( interpolated, axis=0 )
    
    
    diff.append(interpolated)
    diff.append(column)
    df_forecast.append(diff)

df_forecast = pd.DataFrame.from_records(df_forecast)    

In [80]:
len(df_forecast[0][1])

1887

In [92]:
df_forecast=df_forecast.set_index(1)
df_forecast

,0
1,
000221760018,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
000221760021,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
000221760022,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
000221760050,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
000221762018,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...
zz000zzzae20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
zz000zzzfs20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
zz000zzzim20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [93]:
columnlist = [i for i in range (1888)]

In [94]:
dffo = df_forecast.copy()

In [95]:
df_forecast[[columnlist]] = pd.DataFrame(df_forecast[0].tolist(), index= df_forecast.index)

ValueError: Columns must be same length as key

In [ ]:
import re



In [1]:
columnlist = [i for i in range(0,63)]
df_forecast.columns = columnlist
ts_code = df_forecast.index.astype('category').codes
ts_code
total=len(df_forecast.columns)
ind = int(total*0.82)

ind
df_train = df_forecast.iloc[:, :ind].values
df_test = df_forecast.iloc[:, ind:].values

0.001

In [ ]:
freq ='1D'
start_train = pd.Timestamp('2016-01-01', freq=freq)
start_test = pd.Timestamp('2020-04-30', freq=freq)
prediction_length = 12

estimator = DeepAREstimator(freq=freq, 
                           context_length=51,
                           use_feat_static_cat=True,
                           cardinality='auto',
                           num_layers=4, 
                           num_cells=32, 
                           cell_type ='lstm', trainer=Trainer(epochs=50), prediction_length=prediction_length)

In [ ]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

train_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_train,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_train[2000:2500],
                            ts_code[2000:2500].reshape(-1,1))
], freq=freq)



In [ ]:
test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_test,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_test[2000:2500],
                            ts_code[2000:2500].reshape(-1,1))
], freq=freq)




In [18]:
sub

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
t,,,,,,,,,,,,,,,,,,,,,
30036648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-18.95,0.00,0.00,0.00,0.00,0.00,0.00,2735.53,14483.42,1267.40
30036650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12887.87,2272.86,1771.87,3241.33,9965.72,3440.00,1881.96,0.00,0.00,0.00
30036651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,318.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30036652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26080.00,30356.22,10024.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30036653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,603.75
30036654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30036655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,2097.24,0.00,0.00,0.00,0.00
30036656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30036657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,-164.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00
